In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.6 MB/s eta 0:00:00:00:0100:01


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# 模型名称
model_name = "Qwen/Qwen1.5-7B-Chat"

In [4]:
# 配置4-bit量化
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                  # 启用4-bit量化
    bnb_4bit_compute_dtype=torch.float16,  # 计算时用float16加速
    bnb_4bit_quant_type="nf4",          # 量化类型（推荐nf4）
    bnb_4bit_use_double_quant=True,     # 二次量化进一步压缩
)

In [5]:
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",                 # 自动分配 GPU/CPU
    quantization_config=quantization_config,
    trust_remote_code=True,
    torch_dtype=torch.float16,         # 显存优化
)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

2025-07-24 15:37:03.964962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753371424.324639      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753371424.432764      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [7]:
# 推理函数（支持多轮对话）
def generate_response(prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,              # 控制随机性（0~1）
        top_p=0.9,                     # 核采样（0~1）
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
# 测试对话
prompt = "你好，请用Python写一个快速排序算法。"
response = generate_response(prompt, 500)
print("模型回答：\n", response)

模型回答：
 你好，请用Python写一个快速排序算法。 当然，以下是使用Python实现的快速排序算法：

```python
def quick_sort(arr):
    if len(arr) <= 1:
        # 如果数组只有一个元素或为空，直接返回
        return arr
    else:
        # 选择一个基准元素（这里选择第一个元素）
        pivot = arr[0]
        # 创建两个列表，一个用于存放比基准小的元素，另一个存放比基准大的元素
        less_than_pivot = []
        greater_than_pivot = []
        for i in range(1, len(arr)):
            # 如果元素小于基准，放入less_than_pivot列表
            if arr[i] < pivot:
                less_than_pivot.append(arr[i])
            # 否则，放入greater_than_pivot列表
            else:
                greater_than_pivot.append(arr[i])
        # 对两个子列表进行递归排序，然后将结果和基准元素合并
        return quick_sort(less_than_pivot) + [pivot] + quick_sort(greater_than_pivot)

# 测试代码
arr = [3,6,8,10,1,2,1]
print("原始数组:", arr)
print("排序后的数组:", quick_sort(arr))
```

这个快速排序算法首先检查数组的长度，如果长度小于等于1，它就直接返回数组（因为长度为1或0的数组已经是排序好的）。否则，它选择一个基准元素（这里选择第一个元素），并将数组分为两部分：一部分是小于基准的所有元素，另一部分是大于基准的所有元素。然后对这两部分递归地应用同样的过程，最后将结果合并在一起。
